# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-02-08 09:58:35,158 fedbiomed INFO - Component environment:
2022-02-08 09:58:35,160 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-02-08 09:58:36,392 fedbiomed INFO - Messaging researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x10b3aba30>
2022-02-08 09:58:36,424 fedbiomed INFO - Listing available datasets in all nodes... 
2022-02-08 09:58:36,483 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / DEBUG - Message received: {'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'command': 'list'}
2022-02-08 09:58:36,492 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / DEBUG - Message received: {'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'command': 'list'}
2022-02-08 09:58:46,454 fedbiomed INFO - 
 Node: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 | Number of Datasets: 1 
+--------+-------------+-----

{'node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}],
 'node_30b30277-670c-4f99-9772-7b4bfc231b2d': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default)
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.torchnn_refactor import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine 
from copy import deepcopy

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "import torch.nn.functional as F",
                "from torch.utils.data import DataLoader",
                "from opacus import PrivacyEngine",
                "from copy import deepcopy",]
        self.add_dependency(deps)
        
        self.model = self.make_model()
        
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']
        
    def make_model(self):
        model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 3, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.25),
                                  nn.Flatten(),
                                  nn.Linear(9216, 128),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(128, 10),
                                  nn.LogSoftmax(dim=1))
        return model
        
    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss
    
    def preprocess(self):
        # enter PrivacyEngine
        privacy_engine = PrivacyEngine()
        self.model, self.optimizer, self.data = privacy_engine.make_private(module=self.model,
                                                                    optimizer=self.optimizer,
                                                                    data_loader=self.data,
                                                                    noise_multiplier=self.noise_multiplier,
                                                                    max_grad_norm=self.max_grad_norm,
                                                                    )
        
    def postprocess(self, params):
        # params keys are changed by the privacy engine (as _module.param_key): should be re-named
        params_keys = list(params.keys())
        for key in params_keys:
            if '_module' in key:
                newkey = key.replace('_module.', '')
                params[newkey] = params.pop(key)
        return params

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpyu0hz81t/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-02-08 09:59:26,786 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-02-08 09:59:26,829 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / DEBUG - Message received: {'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-02-08 09:59:26,836 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / DEBUG - Message received: {'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-02-08 09:59:36,792 fedbiomed INFO - Node selected for training -> node_30b30277-670c-4f99-9772-7b4bfc231b2d
2022-02-08 09:59:36,793 fedbiomed INFO - Node selected for training -> node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6
2022-02-08 09:59:36,796 fedbiomed INFO - Checking data quality of federated datasets...
2022-02-08 09:59:37,527 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-02-08 10:02:03,023 fedbiomed INFO - Sampled nodes in round 0 ['node_30b30277-670c-4f99-9772-7b4bfc231b2d', 'node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6']
02/08/2022 10:02:03:INFO:Sampled nodes in round 0 ['node_30b30277-670c-4f99-9772-7b4bfc231b2d', 'node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6']
2022-02-08 10:02:03,025 fedbiomed INFO - Send message to node node_30b30277-670c-4f99-9772-7b4bfc231b2d - {'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'job_id': '6028c992-1b77-44e9-a93f-b7c7b7f871e0', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/02/08/my_model_ca8063a6-9d49-4583-bc32-6fc4f30eb490.py', 'params_url': 'http://localhost:8844/media/uploads/2022/02/08/aggregated_params_init_b070b8ef-26eb-4676-9ad8-513bdac909e6.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'n

2022-02-08 10:02:03,164 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'job_id': '6028c992-1b77-44e9-a93f-b7c7b7f871e0', 'params_url': 'http://localhost:8844/media/uploads/2022/02/08/aggregated_params_init_b070b8ef-26eb-4676-9ad8-513bdac909e6.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6': ['dataset_1d452ab5-b19e-428a-bafb-e4cb1e3b7965']}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'model_url': 'http://localhost:8844/media/uploads/2022/02/08/my_model_ca8063a6-9d49-4583-bc32-6fc4f30eb490.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
02/08/2022 10:02:03:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_2968f380-ae67-4262-bfa9-eaf836dbfc45', 'job_id': '60

2022-02-08 10:02:58,310 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [3500/60000 (6%)]	Loss: 2.096828
02/08/2022 10:02:58:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [3500/60000 (6%)]	Loss: 2.096828
2022-02-08 10:02:59,334 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [3780/60000 (6%)]	Loss: 2.149423
02/08/2022 10:02:59:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [3780/60000 (6%)]	Loss: 2.149423
2022-02-08 10:03:04,803 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [3920/60000 (6%)]	Loss: 2.171857
02/08/2022 10:03:04:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [3920/60000 (6%)]	Loss: 2.171857
2022-02-08 10:03:05,322 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [3120/60000 (6%)]	Loss: 2.16762

2022-02-08 10:04:46,967 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [8610/60000 (17%)]	Loss: 1.826797
02/08/2022 10:04:46:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [8610/60000 (17%)]	Loss: 1.826797
2022-02-08 10:04:50,185 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [10780/60000 (18%)]	Loss: 1.659691
02/08/2022 10:04:50:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [10780/60000 (18%)]	Loss: 1.659691
2022-02-08 10:04:53,701 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [10780/60000 (18%)]	Loss: 1.554833
02/08/2022 10:04:53:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [10780/60000 (18%)]	Loss: 1.554833
2022-02-08 10:04:56,335 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [10120/60000 (18%)]	L

2022-02-08 10:06:10,524 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [4320/60000 (7%)]	Loss: 1.285384
02/08/2022 10:06:10:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [4320/60000 (7%)]	Loss: 1.285384
2022-02-08 10:06:15,654 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [5390/60000 (9%)]	Loss: 1.361430
02/08/2022 10:06:15:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [5390/60000 (9%)]	Loss: 1.361430
2022-02-08 10:06:16,691 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [4500/60000 (8%)]	Loss: 1.261899
02/08/2022 10:06:16:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [4500/60000 (8%)]	Loss: 1.261899
2022-02-08 10:06:23,818 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [5640/60000 (10%)]	Loss: 1.3738

...

02/08/2022 10:21:34:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [9300/60000 (12%)]	Loss: 1.943252
2022-02-08 10:21:38,290 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [6720/60000 (13%)]	Loss: 1.979651
02/08/2022 10:21:38:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [6720/60000 (13%)]	Loss: 1.979651
2022-02-08 10:21:40,517 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [6720/60000 (13%)]	Loss: 0.793936
02/08/2022 10:21:40:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 1 [6720/60000 (13%)]	Loss: 0.793936
2022-02-08 10:21:44,745 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [10200/60000 (14%)]	Loss: 1.139556
02/08/2022 10:21:44:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 1 [10200/60000 (14%)]	Loss: 1.139556
2022-

02/08/2022 10:22:59:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [1640/60000 (3%)]	Loss: 1.677977
2022-02-08 10:23:03,917 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [2200/60000 (3%)]	Loss: 1.174684
02/08/2022 10:23:03:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [2200/60000 (3%)]	Loss: 1.174684
2022-02-08 10:23:08,653 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [3000/60000 (4%)]	Loss: 1.195297
02/08/2022 10:23:08:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [3000/60000 (4%)]	Loss: 1.195297
2022-02-08 10:23:13,411 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [2350/60000 (4%)]	Loss: 1.026392
02/08/2022 10:23:13:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [2350/60000 (4%)]	Loss: 1.026392
2022-02-08 10:

02/08/2022 10:24:38:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [9310/60000 (15%)]	Loss: 1.258425
2022-02-08 10:24:38,202 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [8280/60000 (14%)]	Loss: 1.413006
02/08/2022 10:24:38:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [8280/60000 (14%)]	Loss: 1.413006
2022-02-08 10:24:44,614 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [11210/60000 (15%)]	Loss: 1.494045
02/08/2022 10:24:44:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 2 [11210/60000 (15%)]	Loss: 1.494045
2022-02-08 10:24:44,723 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [8800/60000 (16%)]	Loss: 1.409318
02/08/2022 10:24:44:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 2 [8800/60000 (16%)]	Loss: 1.409318
2022-

02/08/2022 10:25:59:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [3660/60000 (5%)]	Loss: 0.803687
2022-02-08 10:26:05,484 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [4560/60000 (6%)]	Loss: 0.895350
02/08/2022 10:26:05:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [4560/60000 (6%)]	Loss: 0.895350
2022-02-08 10:26:05,586 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [3850/60000 (6%)]	Loss: 1.575477
02/08/2022 10:26:05:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [3850/60000 (6%)]	Loss: 1.575477
2022-02-08 10:26:11,403 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [4410/60000 (7%)]	Loss: 1.581583
02/08/2022 10:26:11:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [4410/60000 (7%)]	Loss: 1.581583
2022-02-08 10:

02/08/2022 10:27:32:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [11340/60000 (17%)]	Loss: 1.178962
2022-02-08 10:27:32,315 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [8140/60000 (18%)]	Loss: 1.466591
02/08/2022 10:27:32:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [8140/60000 (18%)]	Loss: 1.466591
2022-02-08 10:27:38,054 fedbiomed INFO - log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [11220/60000 (18%)]	Loss: 1.736023
02/08/2022 10:27:38:INFO:log from: node_30b30277-670c-4f99-9772-7b4bfc231b2d / INFO - Train Epoch: 3 [11220/60000 (18%)]	Loss: 1.736023
2022-02-08 10:27:38,191 fedbiomed INFO - log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [9660/60000 (18%)]	Loss: 2.454063
02/08/2022 10:27:38:INFO:log from: node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 / INFO - Train Epoch: 3 [9660/60000 (18%)]	Loss: 2.454063
2022

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6 :    
		rtime_training=470.21 seconds    
		ptime_training=653.26 seconds    
		rtime_total=485.51 seconds
	- node_30b30277-670c-4f99-9772-7b4bfc231b2d :    
		rtime_training=477.55 seconds    
		ptime_training=661.36 seconds    
		rtime_total=485.88 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_1d452ab5-b19e-428a-bafb-e4cb1e3b7965,node_af45f88e-0f66-4995-9a4a-b7e73ad8c5a6,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.2421,...","{'rtime_training': 470.2119353510002, 'ptime_t..."
1,True,,dataset_58b416e1-3564-4ce7-b98d-55399c4fa096,node_30b30277-670c-4f99-9772-7b4bfc231b2d,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.2035,...","{'rtime_training': 477.5547742829999, 'ptime_t..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/aggregated_params_e922d140-73ce-4b16-8264-b1bd2e61951b.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [ ]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

In [ ]:

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))